In [1]:
from transformers import pipeline, set_seed, GPT2Tokenizer, GPT2LMHeadModel, GPT2Tokenizer
from torch import tensor, numel
from bertviz import model_view

set_seed(42)

2024-02-08 19:24:16.395826: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-08 19:24:16.395912: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-08 19:24:16.399573: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-08 19:24:16.567197: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-08 19:24:18.098988: W tensorflow/compiler/

In [2]:
generator = pipeline('text-generation', model = 'gpt2')
generator("Hello, I'm a language model and I", max_length = 30, num_return_sequences = 3)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': "Hello, I'm a language model and I can use it to get from one place to another (as a model to explain the code of Java)."},
 {'generated_text': "Hello, I'm a language model and I want to explore it when it gets released. So, today, I'm going to create a web version"},
 {'generated_text': "Hello, I'm a language model and I can program with no other tool than Javascript! It's great to run many software programs but it's also"}]

In [3]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

'sinan' in tokenizer.get_vocab()

False

In [4]:
tokenizer.convert_ids_to_tokens(tokenizer.encode('Sinan loves a beautiful day'))

['Sin', 'an', 'Ġloves', 'Ġa', 'Ġbeautiful', 'Ġday']

In [6]:
tokenizer.encode(tokenizer.encode('Sinan loves a beautiful day'))

[46200, 272, 10408, 257, 4950, 1110]

In [7]:
encoded = tokenizer.encode('Sinan loves a beautiful day', return_tensors = 'pt')

encoded

tensor([[46200,   272, 10408,   257,  4950,  1110]])

In [8]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')

In [11]:
model

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [9]:
model.transformer.wte(encoded).shape

torch.Size([1, 6, 768])

In [10]:
model.transformer.wpe(tensor([0, 1, 2, 3, 4, 5]).reshape(1, 6)).shape

torch.Size([1, 6, 768])

In [18]:
initial_input = model.transformer.wte(encoded) + model.transformer.wpe(tensor([0, 1, 2, 3, 4, 5]).reshape(1, 6))

initial_input.shape

torch.Size([1, 6, 768])

In [20]:
initial_input = model.transformer.drop(initial_input)
initial_input

tensor([[[ 0.0107, -0.2453,  0.1275,  ..., -0.1969,  0.0006,  0.1539],
         [-0.1013, -0.0894, -0.0378,  ..., -0.0534, -0.0527,  0.0046],
         [-0.0716, -0.1690,  0.0386,  ..., -0.2034, -0.0197, -0.1113],
         [-0.0509, -0.0682,  0.1526,  ...,  0.0527,  0.0912, -0.0455],
         [ 0.0612,  0.1425,  0.1402,  ...,  0.0964,  0.0510,  0.1474],
         [-0.1283, -0.0632,  0.1287,  ..., -0.0907, -0.0655,  0.1085]]],
       grad_fn=<AddBackward0>)

In [21]:
model.lm_head

Linear(in_features=768, out_features=50257, bias=False)

In [22]:
for module in model.transformer.h:
    initial_input = module(initial_input)[0]

initial_input = model.transformer.ln_f(initial_input)

In [25]:
(initial_input == model(encoded, output_hidden_states = True).hidden_states[-1]).all()

tensor(True)

In [26]:
total_params = 0
for param in model.parameters():
    total_params += numel(param)

print(f'Number of params: {total_params:,}')

Number of params: 124,439,808
